<a href="https://colab.research.google.com/github/soumabkargit/intro-to-machine-learning/blob/master/ClassificationTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# import packages
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# load dataset
data = datasets.load_iris()

# separate features and target
X = data.data
y = data.target

# apply one-hot encoding to targets
one_hot_encoder = OneHotEncoder(categories='auto')
encode_categorical = y.reshape(len(y), 1)
y = one_hot_encoder.fit_transform(encode_categorical).toarray()

# split in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

# build the linear model
class LogisticRegressionModel(Model):
  def __init__(self):
    super(LogisticRegressionModel, self).__init__()
    # initialize weight and bias variables
    self.weight = tf.Variable(
        initial_value = tf.random.normal(
            [4, 3], dtype=tf.float64),
        trainable=True)
    self.bias = tf.Variable(initial_value = tf.random.normal(
        [3], dtype=tf.float64), trainable=True)

  def call(self, inputs):
    return tf.add(tf.matmul(inputs, self.weight), self.bias)

model = LogisticRegressionModel()

# parameters
batch_size = 32
learning_rate = 0.1

# use tf.data to batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Accuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.Accuracy(name='test_accuracy')

# use tf.GradientTape to train the model
@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, predictions))
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(tf.argmax(labels,1), tf.argmax(predictions,1))

@tf.function
def test_step(inputs, labels):
  predictions = model(inputs)
  t_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, predictions))

  test_loss(t_loss)
  test_accuracy(tf.argmax(labels,1), tf.argmax(predictions,1))

num_epochs = 1000

for epoch in range(num_epochs):
  for train_inputs, train_labels in train_ds:
    train_step(train_inputs, train_labels)

  for test_inputs, test_labels in test_ds:
    test_step(test_inputs, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'

  if ((epoch+1) % 100 == 0):
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

Epoch 100, Loss: 0.4285290539264679, Accuracy: 82.33036041259766, Test Loss: 0.3979380130767822, Test Accuracy: 85.18421173095703
Epoch 200, Loss: 0.306321382522583, Accuracy: 88.87053680419922, Test Loss: 0.2681294083595276, Test Accuracy: 92.59210205078125
Epoch 300, Loss: 0.25382715463638306, Accuracy: 91.0922622680664, Test Loss: 0.2117917686700821, Test Accuracy: 94.33333587646484
Epoch 400, Loss: 0.22331503033638, Accuracy: 92.203125, Test Loss: 0.1791626214981079, Test Accuracy: 95.09210205078125
Epoch 500, Loss: 0.20293880999088287, Accuracy: 92.86964416503906, Test Loss: 0.15754000842571259, Test Accuracy: 95.99473571777344
Epoch 600, Loss: 0.1881810575723648, Accuracy: 93.41815948486328, Test Loss: 0.14201098680496216, Test Accuracy: 96.66228485107422
Epoch 700, Loss: 0.17690429091453552, Accuracy: 93.84821319580078, Test Loss: 0.13024184107780457, Test Accuracy: 97.13909912109375
Epoch 800, Loss: 0.16795212030410767, Accuracy: 94.17076110839844, Test Loss: 0.1209714487195015